In [17]:
import numpy as np
import pandas as pd
from predictor.test_predictor import LinearRegressionPredictor
from predictor.train_models import train_linear_regression

In [18]:
model = LinearRegressionPredictor()
model.train_and_save_model() 

PANC


KeyboardInterrupt: 

In [6]:
from predictor.utils import stations_list
for station in stations_list:
    print(model.predict(f"../data/restructured_simple/combined/{station}.csv",station))

       YEAR  MONTH  DAY  TMAX  TMIN  PRCP  SNOW  SNWD  TAVG
26475  2024     11   26   NaN   NaN   NaN   0.0   0.0   NaN
26476  2024     11   27   NaN   NaN   NaN   0.0   0.0   NaN
26477  2024     11   28   NaN   NaN   NaN   0.0   0.0   NaN
26478  2024     11   29   NaN   NaN   NaN   0.0   0.0   NaN
26479  2024     11   30   NaN   NaN   NaN   0.0   0.0   NaN


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [16]:
data = pd.read_csv("regression_data_full/PANC.csv")
data.tail(20)

,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
3937,2024.0,10.0,12.0,48.92,50.36,51.98,0.0,221.0,48.02,51.26,...,44.06,31.10,39.56,48.92,29.12,37.94,48.02,32.00,39.92,44.96
3938,2024.0,10.0,13.0,48.02,51.26,57.92,0.0,0.0,48.02,51.26,...,48.92,29.12,37.94,48.02,32.00,39.92,44.96,30.20,35.06,39.02
3939,2024.0,10.0,14.0,48.02,51.26,51.98,0.0,91.0,48.92,51.62,...,48.02,32.00,39.92,44.96,30.20,35.06,39.02,28.22,31.64,35.96
3940,2024.0,10.0,15.0,48.92,51.62,57.92,0.0,15.0,48.92,53.42,...,44.96,30.20,35.06,39.02,28.22,31.64,35.96,22.10,30.38,35.96
3941,2024.0,10.0,16.0,48.92,53.42,57.92,0.0,5.0,46.94,50.72,...,39.02,28.22,31.64,35.96,22.10,30.38,35.96,20.12,27.86,37.94
3942,2024.0,10.0,17.0,46.94,50.72,55.04,0.0,3.0,42.98,49.28,...,35.96,22.10,30.38,35.96,20.12,27.86,37.94,31.10,37.04,46.94
3943,2024.0,10.0,18.0,42.98,49.28,51.98,0.0,46.0,37.04,46.94,...,35.96,20.12,27.86,37.94,31.10,37.04,46.94,39.02,43.34,46.94
3944,2024.0,10.0,19.0,37.04,46.94,59.00,0.0,0.0,37.94,46.04,...,37.94,31.10,37.04,46.94,39.02,43.34,46.94,31.10,39.20,42.08
3945,2024.0,10.0,20.0,37.94,46.04,53.06,0.0,0.0,37.04,46.58,...,46.94,39.02,43.34,46.94,31.10,39.20,42.08,26.24,32.72,33.98
3946,2024.0,10.0,21.0,37.04,46.58,53.06,0.0,0.0,33.98,41.36,...,46.94,31.10,39.20,42.08,26.24,32.72,33.98,28.22,30.74,35.06


In [19]:
from analysis.evaluate_model import evaluate_model_years
from predictor.test_predictor import RidgeRegressionPredictor

In [21]:
evaluate_model_years(2018,2022,RidgeRegressionPredictor("../predictor/models/RidgeRegression/model_full"))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/judydw/Documents/UMich/Fall 2024/STATS 604/Project1/pythonProject/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/p_/wj4jvr5929x__hfp1fb3gy400000gs/T/ipykernel_88862/3752129994.py", line 1, in <module>
    evaluate_model_years(2018,2022,RidgeRegressionPredictor("../predictor/models/RidgeRegression/model_full"))
  File "/Users/judydw/Documents/GitHub/Weather-Forecast/analysis/evaluate_model.py", line 124, in evaluate_model_years
    mse = evaluate_model_year(year, predictor)
  File "/Users/judydw/Documents/GitHub/Weather-Forecast/analysis/evaluate_model.py", line 106, in evaluate_model_year
    mse = evaluate_model_station_year(station, year, predictor)
  File "/Users/judydw/Documents/GitHub/Weather-Forecast/analysis/evaluate_model.py", line 86, in evaluate_model_station_year
    mse = get_mse_station_day(station, year, 